In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install tensorflow_text
import tensorflow_text

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 8)
from  IPython import display

import pathlib
import shutil
import tempfile
!pip install -q git+https://github.com/tensorflow/docs


import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print("Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


In [ ]:
df=pd.read_csv("../input/feedback-prize-effectiveness/train.csv")

In [ ]:
df.head()

Types of Discourse

In [ ]:
df.discourse_type.unique()

Types of discourse effectiveness 

In [ ]:
df.discourse_effectiveness.unique()

In [ ]:
df.shape

max len essay

In [ ]:
int(df.discourse_text.str.len().max())

min len essay

In [ ]:
int(df.discourse_text.str.len().min())

In [ ]:
df['essay_length'] = df.discourse_text.str.split().str.len()

In [ ]:
df['essay_length'] 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.histplot(df['essay_length'] , ax=ax,kde=True,bins=200)
ax.set_xlim(0, 260)
#ax.set_ylim(-0.005, 0.005)
plt.show()


In [ ]:

df.groupby('discourse_type').count()


In [ ]:
df.discourse_effectiveness.unique()

NNLM import

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")

In [ ]:
df.columns

## One Hot Encoding labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 

def Label_encoder(df,enc):
    encoder_df = pd.DataFrame(enc.fit_transform(df[['discourse_effectiveness']]).toarray())
    return encoder_df

#merge one-hot encoded columns back with original DataFrame


#view final df
print(Label_encoder(df,encoder))

## Seperating X and Y

In [ ]:
def Seperate_Train_Text(df):
    X=df[['discourse_text','discourse_type']]
    y=Label_encoder(df,encoder)
    
    return (X,y)

In [ ]:
X,y=Seperate_Train_Text(df)

In [ ]:
print(X.shape)
print(y)

## Splitting for Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.9,stratify=y)

Create tf.data pipeline


In [ ]:
TrainDataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
ValidDataset = tf.data.Dataset.from_tensor_slices((X_valid,y_valid))


In [ ]:
df.discourse_type.unique()

In [ ]:
def embedding_data(dataset):

    def preprocess(text, label=None):
        
        if text[1]=='Lead':
            embedding =embed([text[0]])+0.5
        elif text[1]=='Position':
            embedding =embed([text[0]])+1
        
        elif text[1]=='Claim':
            embedding =embed([text[0]])+1.5
            
        elif text[1]=='Evidence':
            embedding =embed([text[0]])+2
        elif text[1]=='Counterclaim':
            embedding =embed([text[0]])+2.5
        elif text[1]==' Rebuttal':
            embedding =embed([text[0]])+3
        elif text[1]==' Concluding Statement':
            embedding =embed([text[0]])+3.5        
       
            
        else :
            embedding =embed([text[0]])
            
        if label!=None:
        
            P=text[0], label
            return P
        else:
            return text[0]
            
    return dataset.map(preprocess)
        

In [ ]:
TrainData=embedding_data(TrainDataset)
ValidData=embedding_data(ValidDataset)

In [ ]:

TrainData = TrainData.batch(16)
TrainData = TrainData.shuffle(100)
ValidData=ValidData.batch(16)
ValidData=ValidData.shuffle(100)

In [ ]:
module_url="https://tfhub.dev/google/nnlm-en-dim128/2"

In [ ]:
checkpoint_filepath = './tmp/checkpoint'

In [ ]:

def train_and_evaluate_model(module_url, embed_size, name, trainable=True):
    print("Downloading Model ...")
    hub_layer=hub.KerasLayer(module_url,input_shape=[],output_shape=embed_size,dtype=tf.string,trainable=trainable)
    print(" Model Downloaded ")
    model=tf.keras.models.Sequential([
                    hub_layer,
                    #tf.keras.layers.Dense(32,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.Dense(16,activation='relu'),
                    #tf.keras.layers.BatchNormalization(),
                    #tf.keras.layers.Dense(8,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.Dense(100,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.Dense(64,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.BatchNormalization(),
                    #tf.keras.layers.Dropout(0.3)
                    tf.keras.layers.Dense(3,activation='softmax')
                    #tf.keras.layers.Dropout(0.3),
                    #tf.keras.layers.Dense(256,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.Dense(128,kernel_regularizer =tf.keras.regularizers.l2(0.001),activation='relu'),
                    #tf.keras.layers.Dropout(0.3),
                    #tf.keras.layers.Dense(32,kernel_regularizer =tf.keras.regularizers.l2(0.01),activation='relu'),
                    
  ])
    model.compile(tf.keras.optimizers.Adam(0.00005), 'categorical_crossentropy', ['acc'])
    model.summary()
    checkpoint= tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath ,frequency='epoch',save_weights_only=False,save_best_only=True,
                           verbose=1)
    history=model.fit(TrainData,validation_data=ValidData,
                   # validation_data=(valid_df['Tweets'],valid_df['Labels']),
                    epochs=8,verbose=1,
                    callbacks=[checkpoint,
                    tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=2,mode='max'),tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.4,
    patience=1,
    verbose=1,
    mode='min',
    min_delta=0.000001,
    )])

    return (history,model)

In [ ]:
histories={}
histories['nnlm-128'],_model= train_and_evaluate_model(module_url,embed_size=128,name='nnlm-128')

In [ ]:
testdf=pd.read_csv('../input/feedback-prize-effectiveness/test.csv')

In [ ]:
#tf.saved_model.save(_model,'./')
_model.save("./my_model")

In [ ]:
#!pip install --target=./Packages tensorflow_hub

In [ ]:
#TestX=testdf['discourse_text']

In [ ]:
#TestDataset=tf.data.Dataset.from_tensor_slices(TestX)

In [ ]:
#TestDataset=TestDataset.batch(100)

In [ ]:
#preds=_model.predict(TestDataset)

In [ ]:
#dataset_submission = pd.read_csv("../input/feedback-prize-effectiveness/sample_submission.csv")

In [ ]:

#dataset_submission.loc[:,"Ineffective"] = preds[:,0]
#dataset_submission.loc[:,"Adequate"] = preds[:,1]
#dataset_submission.loc[:,"Effective"] = preds[:,2]
#dataset_submission.head()

In [ ]:
#dataset_submission.to_csv('submission.csv',index=None)

In [ ]:
#print("done")

In [ ]:
#dataset_submission.to_csv('submission.csv',index=None)